In [9]:
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [10]:
def read_csv(file,missingValueSymbol):
    #df=pd.read_csv(file,header=None).replace(missingValueSymbol, np.NaN) #si vamos a trabajar con basball_numeric
    df=pd.read_csv(file).replace(missingValueSymbol, np.NaN)
    return df
file="titanic.csv"
#file="baseball_numeric.csv"
#file="airlinetickets.csv"
DataFrame=read_csv(file,"?")
DataFrame
    

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [20]:
def getNearestNeighbor(df,point):    
    originalpoint=df.iloc[point,:]    
    dPrime=pd.get_dummies(df.dropna(axis='columns'))
    Xi=dPrime.iloc[point,:]  
    distList=np.array([])
    for i,rows in dPrime.iterrows():
        if not(i==point) and np.sum(pd.isnull(originalpoint)&pd.isnull(df.iloc[i,:]))==0:
            euclidean=distance.euclidean(rows,Xi)
            distList=np.append(distList,euclidean)
            if distList.min()==euclidean:
                dist,nearestIndex=euclidean,i
    #print("Nearest neighbor point is: "+str(nearestIndex)+" Distance : "+str(euclidean))
    df=pd.DataFrame.copy(df)          
    df.iloc[point,:]=originalpoint.fillna(value=df.iloc[nearestIndex,:])
    return df

getNearestNeighbor(DataFrame.drop(['Name','Ticket'], axis=1),886)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,B51 B53 B55,S
887,888,1,1,female,19.0,0,0,30.0000,B42,S
888,889,0,3,female,NaN,1,2,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,30.0000,C148,C


In [38]:
def impute(file,missingValueSymbol=None):
        df=read_csv(file,missingValueSymbol)
        #df=df.drop(['Name','Ticket'],axis=1) #En caso de que tenga una columna nombre,ya que esta no aporta nada 
                                              #a la distancia,pueden existir otras columnas que tampoco lo hagan
        for i in range(df.shape[0]):
            df=getNearestNeighbor(df,i) 
                   
        return df
#Se demora mucho tiempo en correr ...
impute(file,"?")

KeyboardInterrupt: 

array([[  0.        ,  64.09573706,   2.90785574, ..., 888.15113579,
        889.29441835, 890.00070225],
       [ 64.09573706,   0.        ,  63.40563208, ..., 888.29388413,
        888.96080389, 891.27183295],
       [  2.90785574,  63.40563208,   0.        , ..., 886.13939401,
        887.27803175, 888.00170643],
       ...,
       [888.15113579, 888.29388413, 886.13939401, ...,   0.        ,
          7.47679744,  16.04649494],
       [889.29441835, 888.96080389, 887.27803175, ...,   7.47679744,
          0.        ,  22.38442539],
       [890.00070225, 891.27183295, 888.00170643, ...,  16.04649494,
         22.38442539,   0.        ]])